In [17]:
import pandas as pd
import numpy as np


In [18]:
def clean_data(df):
    # 1. Drop các cột không dùng
    df = df.drop(
        columns=[
            "git_merged_with",
            "tr_log_num_test_suites_run",
            "tr_log_num_test_suites_failed",
            "tr_log_num_test_suites_ok",
            "gh_num_commits_in_push",
            "gh_commits_in_push",
            "gh_first_commit_created_at",
            "tr_virtual_merged_into",
            "gh_pushed_at",
            "tr_log_buildduration",
        ],
        errors="ignore",
    )

    # 3. Loại bỏ status errored và canceled
    df = df[~df["tr_status"].isin(["errored", "canceled"])]

    return df


# Load CSV
df = pd.read_csv(r"/Users/hunglai/hust/20251/thesis/19314170/final-2017-01-25.csv")

# Apply cleaning
df_clean = clean_data(df.copy())

df_clean.head()

KeyboardInterrupt: 

In [ ]:
# Remove not exists projects
df_filtered = df_clean[
    ~df_clean["gh_project_name"].isin(
        [
            "Homebrew/homebrew-science",
            "BBC-News/wraith",
            "Homebrew/homebrew-php",
            "caskroom/homebrew-versions",
            "ets-berkeley-edu/calcentral",
            "apache/sling"
        ]
    )
]

In [ ]:
df_filtered.to_csv("/Users/hunglai/hust/20251/thesis/19314170/filtered_travistorrent.csv", index=False)

In [ ]:
df_filtered = pd.read_csv("/Users/hunglai/hust/20251/thesis/19314170/filtered_travistorrent.csv")
df_filtered.info()

In [ ]:
langs = ["ruby", "python", "java"]
df_filtered = df_filtered[df_filtered["gh_lang"].isin(langs)]

In [ ]:
# 1. Đếm số trigger commit như bạn đang làm
grouped_commits = (
    df_filtered
    .groupby('gh_project_name')['git_trigger_commit']
    .agg(unique_trigger_commits=pd.Series.nunique)
    .reset_index()
)

# 2. Tính tuổi repo theo từng project từ gh_repo_age
age_stats = (
    df_filtered
    .groupby('gh_project_name')['gh_repo_age']
    .agg(gh_repo_age_min='min', gh_repo_age_max='max')
    .assign(
        age_span_days=lambda x: x['gh_repo_age_max'] - x['gh_repo_age_min'],
        age_span_years=lambda x: x['age_span_days'] / 365.0
        
    )
    .reset_index()
)

grouped_commits = (
    grouped_commits
    .merge(age_stats[['gh_project_name', 'age_span_years']],
           on='gh_project_name',
           how='inner')
)

grouped_commits = (
    grouped_commits
    .query('unique_trigger_commits > 800 and age_span_years >= 2')
    .sort_values('unique_trigger_commits', ascending=False)
    .reset_index(drop=True)
)

filtered_df_clean = (
    df_filtered[df_filtered['gh_project_name'].isin(grouped_commits['gh_project_name'])]
    .reset_index(drop=True)
)

filtered_df_clean.head()


In [ ]:
# # 1. Đếm số trigger commit (unique + total)
# grouped_commits = (
#     df_filtered
#     .groupby('gh_project_name')['git_trigger_commit']
#     .agg(
#         unique_trigger_commits=pd.Series.nunique,
#         total_trigger_commits='count'
#     )
#     .reset_index()
# )

# # 2. Tính tuổi repo theo từng project từ gh_repo_age
# age_stats = (
#     df_filtered
#     .groupby('gh_project_name')['gh_repo_age']
#     .agg(gh_repo_age_min='min', gh_repo_age_max='max')
#     .assign(
#         age_span_days=lambda x: x['gh_repo_age_max'] - x['gh_repo_age_min'],
#         age_span_years=lambda x: x['age_span_days'] / 365.0
#     )
#     .reset_index()
# )

# # Merge age_span_years vào grouped_commits
# grouped_commits = grouped_commits.merge(
#     age_stats[['gh_project_name', 'age_span_years']],
#     on='gh_project_name',
#     how='inner'
# )

# # Điều kiện đếm
# commit_thresholds = [200, 400, 600, 800]
# age_thresholds = [1, 2, 3, 4]

# rows = []

# for c in commit_thresholds:
#     for a in age_thresholds:
#         subset = grouped_commits.query(
#             'unique_trigger_commits > @c and age_span_years >= @a'
#         )
#         rows.append({
#             'commit_min': c,
#             'age_min_years': a,
#             'project_count': subset.shape[0],
#             'unique_trigger_commits_sum': subset['unique_trigger_commits'].sum(), 
#         })

# summary = pd.DataFrame(rows)

# summary_pivot = summary.pivot(
#     index='age_min_years',
#     columns='commit_min',
#     values='project_count'
# ).sort_index().sort_index(axis=1)

# print(summary)
# print(summary_pivot)


In [ ]:
# filtered_df_clean = (
#     df_clean[df_clean['gh_project_name'].isin(grouped_commits['gh_project_name'])]
#     .reset_index(drop=True)
# )


In [ ]:
filtered_df_clean.to_csv('/Users/hunglai/hust/20251/thesis/19314170/800c_2y_filtered_travistorrent.csv', index=False)

In [19]:
filtered_df_clean = pd.read_csv('/Users/hunglai/hust/20251/thesis/19314170/800c_2y_filtered_travistorrent.csv')

/var/folders/zq/dgcqd65d2_5dxc9q_nrm0qjr0000gn/T/ipykernel_22363/1315796083.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  filtered_df_clean = pd.read_csv('/Users/hunglai/hust/20251/thesis/19314170/800c_2y_filtered_travistorrent.csv')


In [23]:
def aggregate_jobs_to_builds(df):
    grouped = df.groupby(['tr_build_id', 'tr_original_commit','tr_build_number'])
    agg_dict = {
        'tr_log_num_jobs': grouped.size(),
        'tr_log_tests_run_sum': grouped['tr_log_num_tests_run'].sum(),
        'tr_log_tests_failed_sum': grouped['tr_log_num_tests_failed'].sum(),
        'tr_log_tests_skipped_sum': grouped['tr_log_num_tests_skipped'].sum(),
        'tr_log_tests_ok_sum': grouped['tr_log_num_tests_ok'].sum(),
        'tr_log_testduration_sum': grouped['tr_log_testduration'].sum(),
    }

    df_agg = pd.DataFrame(agg_dict)

    # Fail rate
    df_agg['tr_log_tests_fail_rate'] = np.where(
        df_agg['tr_log_tests_run_sum'] > 0,
        df_agg['tr_log_tests_failed_sum'] / df_agg['tr_log_tests_run_sum'],
        0.0
    )

    # --- NEW: Union all analyzers per build ---
    def union_analyzers(series):
        analyzers = set()
        for s in series.dropna():
            if isinstance(s, str) and s:
                analyzers.update(s.replace(',', ' ').split())
        return ','.join(sorted(analyzers)) if analyzers else ''
    
    df_agg['tr_log_analyzers_all'] = grouped['tr_log_analyzer'].apply(union_analyzers)

    # Union frameworks
    def union_frameworks(series):
        all_frameworks = set()
        for s in series.dropna():
            if isinstance(s, str) and s:
                all_frameworks.update(s.replace(',', ' ').split())
        return ','.join(sorted(all_frameworks)) if all_frameworks else ''
    
    df_agg['tr_log_frameworks_all'] = grouped['tr_log_frameworks'].apply(union_frameworks)

    # Union languages
    def union_languages(series):
        all_languages = set()
        for s in series.dropna():
            if isinstance(s, str) and s:
                all_languages.update(s.replace(',', ' ').split())
        return ','.join(sorted(all_languages)) if all_languages else ''
    
    df_agg['tr_log_lan_all'] = grouped['tr_log_lan'].apply(union_languages)

    # Keep representative columns
    first_job_cols = [
        'tr_duration',
        'tr_status',
        'tr_jobs'
    ]
    for col in first_job_cols:
        if col in df.columns:
            df_agg[col] = grouped[col].first()

    return df_agg.reset_index()

# 4. Aggregate build-level logs

df_builds = aggregate_jobs_to_builds(filtered_df_clean)
print(f"Aggregated builds: {len(df_builds)}")

Aggregated builds: 470007


In [24]:
# 5. Merge back general build info (not job-specific)
build_info_cols = [
    col
    for col in filtered_df_clean.columns
    if col
    not in [
        "tr_job_id",
        "tr_log_num_tests_ok",
        "tr_log_num_tests_failed",
        "tr_log_num_tests_run",
        "tr_log_num_tests_skipped",
        "tr_log_tests_failed",
        "tr_log_testduration",
        "tr_log_setup_time",
        "tr_log_analyzer",
        "tr_log_frameworks",
        "tr_log_status",
        "tr_log_bool_tests_ran",
        "tr_log_bool_tests_failed",
        "tr_jobs",
        "tr_status",
        "tr_duration",
        "tr_log_lan",
    ]
]

df_build_info = (
    filtered_df_clean[build_info_cols]
    .groupby(["tr_build_id", "tr_original_commit", "tr_build_number"])
    .first()
    .reset_index()
)

df_builds_final = df_build_info.merge(
    df_builds, on=["tr_build_id", "tr_original_commit", "tr_build_number"], how="left"
)


df_builds_final = df_builds_final.sort_values(
    by=["gh_project_name", "tr_build_id", "tr_original_commit"],
    ascending=[True, True, True],
).reset_index(drop=True)

print(f"✅ Final build dataset shape: {df_builds_final.shape}")

df_builds_final.head(10)

✅ Final build dataset shape: (470007, 52)


,tr_build_id,tr_original_commit,tr_build_number,gh_project_name,gh_is_pr,gh_pr_created_at,gh_pull_req_num,gh_lang,git_branch,git_prev_commit_resolution_status,...,tr_log_tests_skipped_sum,tr_log_tests_ok_sum,tr_log_testduration_sum,tr_log_tests_fail_rate,tr_log_analyzers_all,tr_log_frameworks_all,tr_log_lan_all,tr_duration,tr_status,tr_jobs
0,223084,dfcbe784a598382625a2da337613da04b73785d5,1,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,master,merge_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,23.0,passed,[223085]
1,223093,83ca85f58495cad524ec70198f9d422ff95ab3b4,2,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,merge_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,134.0,failed,[223094]
2,223126,6df541d5b0c8339af0a3e894bb4aac7ca0b0a795,3,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,build_found,...,2.0,24.0,7.68,0.142857,ruby,rspec,ruby,152.0,failed,[223127]
3,223161,a781d177ff49a54a78cd22f31234b1bc18b87fca,4,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,build_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,151.0,failed,[223162]
4,223171,591ecd66aa5af727acff7de75205cdf2f8609202,5,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,build_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,154.0,failed,[223172]
5,223174,f4e0ef535e0bfc374bbfc4a4009635a46675ac0f,6,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,build_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,150.0,failed,[223175]
6,223181,89f366ecf9d79866d4b36894fe795e0a8bfed390,7,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,build_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,145.0,failed,[223182]
7,223943,b54539fbab29b9ac06980d116e7dfb616ff29cdc,8,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,build_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,136.0,failed,[223944]
8,224639,f340856c10ad9e166b14fc4e50f6b7ea77127f9d,10,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,build_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,153.0,failed,[224640]
9,225324,f87ba60bd92e62930c9ce86a0e7911ad5908c3ea,11,AlchemyCMS/alchemy_cms,False,None,NaN,ruby,next_stable,build_found,...,0.0,0.0,0.00,0.000000,ruby,,ruby,146.0,failed,[225325]


In [25]:
df_builds_final.to_csv(r'/Users/hunglai/hust/20251/thesis/19314170/final-2017-01-25.800c.2y.all.builds.csv', index=False)